# Linkedin Scrapping Profile

### Install Library and Import

In [5]:
pip install selenium webdriver_manager beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import os
from configparser import ConfigParser
import time
import random

### Set up the webdriver and login information

In [7]:
config = ConfigParser()
config.read('config.ini')
username = config['LINKEDIN']['username']
password = config['LINKEDIN']['password']

In [8]:
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.linkedin.com/login")

driver.find_element(By.ID, "username").send_keys(username)
driver.find_element(By.ID, "password").send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()

WebDriverWait(driver, 10).until(EC.url_contains("feed"))

True

### helper function

In [9]:
#scroll down to add human-like interaction and load all the posts
def scroll_down(n, pixels):
    # Scroll down 500 pixels up to 30 times or until no more scrolling is possible
    for _ in range(n):
        driver.execute_script(f"window.scrollBy(0, {random.randint(pixels-100,pixels+100)});")
        time.sleep(3)  # Wait for 3 seconds for new content to load

        # Check if we can scroll further
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height <= driver.execute_script("return window.scrollY + window.innerHeight"):
            break  # Break if no more scrolling is possible

# Create the folder if it doesn't exist
folder_path="csv_data"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

### Get the profile headline

In [10]:
URL_PROFILE = "https://www.linkedin.com/in/sayyidan-i/"
driver.get(URL_PROFILE)

In [11]:
# Profile Headline
name_element = driver.find_element(By.XPATH, "//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']")
headline_element = driver.find_element(By.XPATH, "//div[@class='text-body-medium break-words']")
connections_element = driver.find_element(By.XPATH, "//span[@class='t-bold']")


# Extract and print the connection number
name = name_element.text.strip()
headline = headline_element.text.strip()
connections = connections_element.text.strip()

print(f"Name: {name}")
print(f"Headline: {headline}")
print(f"Connections: {connections}")

Name: Sayyidan Muhamad Ikhsan
Headline: Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast
Connections: 221


In [12]:
# Get the page source
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Initialize a variable to store the skills data
about = ""

# Find the div that contains the skills information
about_div = soup.find('div', class_='SXjmahSTOQiuxpKMNmYxiEpFsQMjwfVyogEk inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp full-width')

if about_div:
    # Extract text from the visually-hidden span
    hidden_span = about_div.find('span', class_='visually-hidden')
    if hidden_span:
        about = hidden_span.get_text(strip=True)
    else:
        about = "Skills not found"
else:
    about = "Skills div not found"

# Print the extracted skills data
print(about)

I am Sayyidan Muhamad Ikhsan, a passionate and versatile final-year student at Universitas Gadjah Mada, blending a mosaic of experiences in various fields of organizational leadership to entrepreneurial ventures and community service. Driven by an unwavering passion for artificial intelligence, I continue to learn and adapt seamlessly to dynamic challenges. My journey has shaped a versatile professional characterized by adaptability, a strong work ethic, and insatiable curiosity. With a high sense of responsibility, I consistently strive for excellence - in both independent and team efforts. I am ready to channel my adaptability, passion for continuous learning, and deep interest in artificial intelligence into meaningful contributions across various roles and projects.


In [13]:
# Create a DataFrame with the extracted profile information
profile_data = {
    'Name': [name],
    'Headline': [headline],
    'Connections': [connections],
    'About': [about]
}

df_profile = pd.DataFrame(profile_data)

# Export the DataFrame to a CSV file
df_profile.to_csv('csv_data\linkedin_profile.csv', index=False)

### Get the activity

In [14]:
# Open the LinkedIn profile activity page
ACTIVITY_URL = f'{URL_PROFILE}detail/recent-activity/'
driver.get(ACTIVITY_URL)

# Scroll down 500 pixels up to 30 times or until no more scrolling is possible
scroll_down(30, 500)

# Get the page source after scrolling
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Prepare a list to collect post data
posts_data = []

# Find all posts
posts = soup.find_all("div", class_="display-flex flex-column flex-grow-1")

# Collect the required information from each post
for post in posts:
    # Initialize a dictionary to hold post data
    post_info = {}

    # Check for repost status
    repost_check = post.find("span", class_="update-components-header__text-view")
    post_info['Is Reposted'] = repost_check and "reposted this" in repost_check.get_text(strip=True)

    # Extract time posted from visually hidden span
    time_posted = post.find("span", class_="update-components-actor__sub-description t-12 t-normal t-black--light")
    hidden_time = time_posted.find("span", class_="visually-hidden")
    post_info['Time Posted'] = hidden_time.get_text(strip=True) if hidden_time else None

    # Extract post caption
    post_caption = post.find("div", class_="update-components-text relative update-components-update-v2__commentary")
    post_info['Post Caption'] = post_caption.get_text(strip=True) if post_caption else None

    # Extract reaction count
    reaction_span = post.find("span", class_="social-details-social-counts__reactions-count")
    reaction_count = reaction_span.get_text(strip=True) if reaction_span else '0'
    post_info['Reaction Count'] = int(reaction_count.replace(',', ''))  # Remove commas before converting

    # Extract comment count
    comment_button = post.find("button", aria_label=True)
    comment_count = comment_button['aria-label'].split()[0] if comment_button else '0'
    post_info['Comment Count'] = int(comment_count.replace(',', ''))  # Remove commas before converting

    # Append the post information to the list
    posts_data.append(post_info)

if not posts_data:
    print("No posts found on the activity page")

# Create a DataFrame from the collected data
df_activity = pd.DataFrame(posts_data)

# Write the DataFrame to a CSV file
df_activity.to_csv('csv_data\linkedin_posts.csv', index=False, encoding='utf-8')

### Get the experience

In [15]:
#open the LinkedIn profile experience page
URL_EXP = f'{URL_PROFILE}details/experience/'

# Your previous code to load the page
driver.get(URL_EXP)
time.sleep(5)  # Allow the page to load
scroll_down(30, 500)

# Get the page source after loading
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Find all experience containers
experience_list = soup.find_all('li', class_='pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column')

# Initialize a list to store the extracted data
experiences = []

# Iterate over each experience container
for experience in experience_list:
    # Try to find the position div with class "mr1 t-bold"
    position_div = experience.find('div', class_='display-flex align-items-center mr1 t-bold')

    if position_div:
        # Extract the single position from the visually-hidden span
        position_span = position_div.find('span', class_='visually-hidden')
        if position_span:
            position = position_span.get_text(strip=True)
        
        # Get status
        status_div = experience.find('span', class_='t-14 t-normal')
        if status_div:
            status_span = status_div.find('span', class_='visually-hidden')
            status = status_span.get_text(strip=True) if status_span else "Status not found"
        
        # Get time information
        time_div = experience.find('span', class_='pvs-entity__caption-wrapper')
        if time_div:
            time_info = time_div.get_text(strip=True)
        else:
            time_info = "Time not found"
        
        # Get caption data from visually-hidden span
        caption_div = experience.find('div', class_='display-flex align-items-center t-14 t-normal t-black')
        if caption_div:
            caption_span = caption_div.find('span', class_='visually-hidden')
            caption = caption_span.get_text(strip=True) if caption_span else "Caption not found"
            # Clean the caption by replacing new lines with dots
            caption = caption.replace('\n', '. ').replace('\r', '')  # Replace new lines with dots
        else:
            caption = "Caption not found"
        
        # Add the data to the list without location
        experiences.append({
            'Position': position,
            'Status': status,
            'Time': time_info,
            'Caption': caption
        })
    
    else:
        # Handle the case where there are multiple positions under one location
        multiple_positions_divs = experience.find_all('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold')
        
        # Extract the relevant status from the custom class for multiple positions
        location_status_div = experience.find('div', class_='fNWbxLGnqELwvLQkpFOBqndmvYRoRbCrA GZxrfCTVzkMgqhWuZbUiEolViXjfRzCMeTUvwKA')
        if location_status_div:
            # Extract the visually-hidden span text for status
            status_div = location_status_div.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold')
            if status_div:
                status_span = status_div.find('span', class_='visually-hidden')
                status = status_span.get_text(strip=True) if status_span else "Status not found"
        
            # Get time information
            time_div = location_status_div.find('span', class_='pvs-entity__caption-wrapper')
            if time_div:
                time_info = time_div.get_text(strip=True)
            else:
                time_info = "Time not found"
        
        # Skip the first div (location), and create separate rows for each position
        for index, position_div in enumerate(multiple_positions_divs):
            if index == 0:
                continue  # Skip the first entry (location)
            
            # Extract the visually-hidden span text
            position_span = position_div.find('span', class_='visually-hidden')
            if position_span:
                position = position_span.get_text(strip=True)
                
                # Add the data to the list without location
                experiences.append({
                    'Position': position,
                    'Status': status,
                    'Time': time_info,
                    'Caption': caption
                })


if not experiences:
    experiences.append({'Position': 'Experiences not found', 'Status': 'Status not found', 'Time': 'Time not found', 'Caption': 'Caption not found'})

# Create a DataFrame from the experiences list
df_experiences = pd.DataFrame(experiences)
print(df_experiences)

# Export the DataFrame to a CSV file
df_experiences.to_csv('csv_data\experience_details.csv', index=False, encoding='utf-8')


                                            Position  \
0                 Accelerator Startup Program Intern   
1                            Machine Learning Cohort   
2               Electromedicine Assistant Internship   
3                Staff of Kementrian Ekonomi Kreatif   
4              Publication at Pekan Wirausaha Teknik   
5                                   Staff of Adkesma   
6  Staff of Consumption, Logistics, and Transport...   
7                               Freelance Math Tutor   
8                           Staff of Public Relation   
9           Video Production Division of Teknik Fair   

                                              Status  \
0                         Indigo Telkom · Internship   
1  Bangkit Academy led by Google, Tokopedia, Goje...   
2  Medika Plaza (PT Kartika Bina Medikatama) · In...   
3                Staff of Kementrian Ekonomi Kreatif   
4                Staff of Kementrian Ekonomi Kreatif   
5                                      KMTETI F

### Get the education

In [16]:
# open volunteer page
URL_education = f'{URL_PROFILE}details/education/'
driver.get(URL_education)
time.sleep(5)  # Allow the page to load
scroll_down(30, 500)

# Get the page source after loading
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Initialize an empty list for education details
education_details = []

# Find the education container
education_container = soup.find('div', class_='scaffold-finite-scroll__content')

# Extract each education entry
if education_container:
    education_entries = education_container.find_all('li', class_='pvs-list__paged-list-item')  # Adjust if necessary

    for entry in education_entries:
        # Get university information
        university_div = entry.find('div', class_='display-flex align-items-center mr1 hoverable-link-text t-bold')
        university = university_div.find('span', class_='visually-hidden').get_text(strip=True) if university_div else "University not found"
        
        # Get field of study
        field_div = entry.find('span', class_='t-14 t-normal')
        field = field_div.find('span', class_='visually-hidden').get_text(strip=True) if field_div else "Field not found"
        
        # Get description
        description_div = entry.find('div', class_='display-flex align-items-center t-14 t-normal t-black')
        description = description_div.find('span', class_='visually-hidden').get_text(strip=True) if description_div else "Description not found"
        
        # Append to the education details list
        education_details.append({
            'University': university,
            'Field': field,
            'Description': description
        })

# If no education details were found, add a default entry
if not education_details:
    education_details.append({'University': 'Education not found', 'Field': 'Field not found', 'Description': 'Description not found'})

# Create a DataFrame from the education details list
df_education = pd.DataFrame(education_details)
print(df_education)

# Export the DataFrame to a CSV file
df_education.to_csv('csv_data\education_details.csv', index=False, encoding='utf-8')


            University            Field            Description
0  Education not found  Field not found  Description not found


### Get Project Information

In [17]:
# Open the LinkedIn profile projects page
PROJECTS_URL = f'{URL_PROFILE}details/projects/'
driver.get(PROJECTS_URL)
time.sleep(5)  # Allow the page to load
scroll_down(30, 500)

# Get the page source after loading
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Find the outer div that contains all projects
projects_div = soup.find('div', class_='scaffold-finite-scroll scaffold-finite-scroll--infinite')
#print(projects_div)

# Find all project name divs within that outer div
if projects_div:
    project_divs = projects_div.find_all('div', class_='display-flex align-items-center mr1 t-bold')
    # Initialize a list to store project details
    
    
    projects = []

    for project_div in project_divs:
        # Extract the project name
        project_name = project_div.get_text(strip=True)

        # Find the time span related to this project
        time_span = project_div.find_next('span', class_='t-14 t-normal')
        if time_span is not None:
            date_info = time_span.find('span', class_='visually-hidden')
            if date_info:
                time_data = date_info.get_text(strip=True)
            else:
                time_data = "Date not found"
        else:
            time_data = "Time span not found"

        # Find the caption for this project
        caption_div = project_div.find_next('li', class_='pvs-list__item--with-top-padding huOBYMIdZizXtwNCBcwmBCRytomUXHoLNnetHMA')
        if caption_div:
            caption_info = caption_div.find('span', class_='visually-hidden')
            if caption_info:
                caption_data = caption_info.get_text(strip=True)
                caption_data = caption_data.replace('\n', '. ')
            else:
                caption_data = "Caption not found"
        else:
            caption_data = "Caption div not found"

        # Append the project details to the projects list
        projects.append({
            'name': project_name,
            'date': time_data,
            'caption': caption_data
        })

if not projects:
    projects.append({'name': 'Projects not found', 'date': 'Date not found', 'caption': 'Caption not found'})

# Create a DataFrame from the projects list
df_projects = pd.DataFrame(projects)
print(df_projects)

# Export the DataFrame to a CSV file
df_projects.to_csv('csv_data\projects_details.csv', index=False, encoding='utf-8')

                 name            date            caption
0  Projects not found  Date not found  Caption not found


### Get Volunteer Experience

In [18]:
# open volunteer page
URL_VOLUNTEER = f'{URL_PROFILE}details/volunteering-experiences/'
driver.get(URL_VOLUNTEER)
time.sleep(5)  # Allow the page to load
scroll_down(30, 500)

# Get the page source after loading
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Find the outer div that contains all projects
volunteer_div = soup.find('div', class_='scaffold-finite-scroll scaffold-finite-scroll--infinite')
#print(volunteer_div)

volunteer = []
# Find all project name divs within that outer div
if volunteer_div:
    volunteer_divs = volunteer_div.find_all('div', class_='display-flex align-items-center mr1 t-bold')
    # Initialize a list to store project details

    for volunteer_div in volunteer_divs:
        # Extract the project name
        volunteer_name = volunteer_div.get_text(strip=True)

        # Find the time span related to this project
        time_span = volunteer_div.find_next('span', class_='t-14 t-normal')
        if time_span is not None:
            date_info = time_span.find('span', class_='visually-hidden')
            if date_info:
                time_data = date_info.get_text(strip=True)
            else:
                time_data = "Date not found"
        else:
            time_data = "Time span not found"

        # Find the description for this project
        description_div = volunteer_div.find_next('li', class_='pvs-list__item--with-top-padding huOBYMIdZizXtwNCBcwmBCRytomUXHoLNnetHMA')
        if description_div:
            description_info = description_div.find('span', class_='visually-hidden')
            if description_info:
                description_data = description_info.get_text(strip=True)
                description_data = description_data.replace('\n', '. ')
            else:
                description_data = "description not found"
        else:
            description_data = "description div not found"

        # Append the project details to the volunteer list
        volunteer.append({
            'name': volunteer_name,
            'date': time_data,
            'description': description_data
        })

if not volunteer:
    volunteer.append({'name': 'volunteer not found', 'date': 'Date not found', 'description': 'description not found'})

# Create a DataFrame from the volunteer list
df_volunteer = pd.DataFrame(volunteer)
print(df_volunteer)

# Export the DataFrame to a CSV file
df_volunteer.to_csv('csv_data/volunteer_details.csv', index=False, encoding='utf-8')



                                       name  \
0      Data Collector IGDData Collector IGD   
1      Education CampaignEducation Campaign   
2  Mental Health RangerMental Health Ranger   

                                   date            description  
0                           Indorelawan  description not found  
1           Character Matters Indonesia  description not found  
2  Satu Persen - Indonesian Life School  description not found  


### Get License and Certification

In [19]:
# open volunteer page
URL_LICENSES = f'{URL_PROFILE}details/certifications/'
driver.get(URL_LICENSES)
time.sleep(5)  # Allow the page to load
scroll_down(30, 500)


# Get the page source after loading
page_source = driver.page_source

# Use Beautiful Soup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')

# Find the outer div that contains all projects
licenses_div = soup.find('div', class_='scaffold-finite-scroll scaffold-finite-scroll--infinite')
#print(volunteer_div)

licenses = []
# Find all project name divs within that outer div
if licenses_div:
    licenses_divs = licenses_div.find_all('div', class_='display-flex flex-wrap align-items-center full-height')
    # Initialize a list to store project details

    for licenses_div in licenses_divs:
        # Extract the project name
        licenses_name = licenses_div.get_text(strip=True)

        # Find the time span related to this project
        organization = licenses_div.find_next('span', class_='t-14 t-normal')
        if organization:
            organization_info = organization.find('span', class_='visually-hidden')
            if organization_info:
                organization_data = organization_info.get_text(strip=True)
            else:
                organization_data = "Organization not found"
            
        # Find the issued date for every license
        issued_date = licenses_div.find_next('span', class_='pvs-entity__caption-wrapper')
        if issued_date:
            issued_date_info = issued_date.get_text(strip=True)
        else:
            issued_date_info = "Issued date not found"

        # Append the project details to the licenses list
        licenses.append({
            'name': licenses_name,
            'organization': organization_data,
            'issued_date': issued_date_info
        })

if not licenses:
    licenses.append({'name': 'licenses not found', 'organization': 'Date not found'})

# Create a DataFrame from the licenses list
df_licenses = pd.DataFrame(licenses)
print(df_licenses)
# Export the DataFrame to a CSV file
df_licenses.to_csv('csv_data\licenses_details.csv', index=False, encoding='utf-8')

                                                 name          organization  \
0   Advanced Learning AlgorithmsAdvanced Learning ...       DeepLearning.AI   
1   Introduction to TensorFlow for Artificial Inte...       DeepLearning.AI   
2   Supervised Machine Learning: Regression and Cl...       DeepLearning.AI   
3   Unsupervised Learning, Recommenders, Reinforce...       DeepLearning.AI   
4   Analyze Data to Answer QuestionsAnalyze Data t...                Google   
5   Ask Questions to Make Data-Driven DecisionsAsk...                Google   
6   Calculus for Machine Learning and Data Science...       DeepLearning.AI   
7   Foundations: Data, Data, EverywhereFoundations...                Google   
8   Linear Algebra for Machine Learning and Data S...       DeepLearning.AI   
9   Prepare Data for ExplorationPrepare Data for E...                Google   
10  Probability & Statistics for Machine Learning ...       DeepLearning.AI   
11  Process Data from Dirty to CleanProcess Data f..

# Ask LLM to Roast LinkedIn Data

Now we have several LinkedIn data, including 

- Headline
- Activity
- Experiences
- Projects
- Volunteer
- Licenses

## ChatGPT API

### Import Libraries

In [20]:
from openai import OpenAI
from IPython.display import Markdown, display

def print_markdown(md_string):
    display(Markdown(md_string))

### Convert Data Frame to CSV-like String

In [21]:
# convert data frame into csv-like string
profile_csv = df_profile.to_csv()
activity_csv = df_activity.to_csv()
experience_csv = df_experiences.to_csv()
education_csv = df_education.to_csv()
projects_csv = df_projects.to_csv()
volunteer_csv = df_volunteer.to_csv()
licenses_csv = df_licenses.to_csv()


In [12]:
# If you want to load data from CSV files
profile_csv = open('csv_data/linkedin_profile.csv', 'r', encoding='utf-8').read()
activity_csv = open('csv_data/linkedin_posts.csv', 'r', encoding='utf-8').read()
experience_csv = open('csv_data/experience_details.csv', 'r', encoding='utf-8').read()
education_csv = open('csv_data/education_details.csv', 'r', encoding='utf-8').read()
projects_csv = open('csv_data/projects_details.csv', 'r', encoding='utf-8').read()
volunteer_csv = open('csv_data/volunteer_details.csv', 'r', encoding='utf-8').read()
licenses_csv = open('csv_data/licenses_details.csv', 'r', encoding='utf-8').read()

### Create Specific System Prompt

In [13]:
DEFAULT_SYSTEM_PROMPT = """

You are a chatbot designed to humorously roast users’ LinkedIn profiles, giving them a fun, spicy critique. You will receive various sections of their profile such as personal details, activity, experience, education, projects, volunteer work, and licenses. Your tone should be witty, sarcastic, and casual, aiming to give lighthearted feedback that helps users upgrade their profiles, but with a sharp edge of humor. Here’s how you should approach each section:

Profile:

If their headline or summary is generic, bland, or confusing, roast it for being forgettable or corporate-speak nonsense.
If the profile picture is missing or unprofessional, call it out with witty remarks like, "Missing a profile pic? What’s the matter, are you a ghost or just camera shy?"
If their headline is unimpressive, roast it for sounding like they threw buzzwords into a blender.
Activity:

If they rarely engage or post, sarcastically applaud them for being “a shadow lurking in the LinkedIn abyss.”
If they spam posts with motivational quotes, call them out for being LinkedIn’s version of a fortune cookie.
Roast excessive likes and comments on "Congrats on the promotion" posts with a "Gold star for you, cheerleader of the year!"
Experience:

If their job descriptions are vague or full of jargon, mock them for sounding like a robot wrote it.
If they have gaps in their work history, joke about them time-traveling or joining a secret spy agency.
If they've stayed in one position for too long without growth, nudge them by saying, "Comfy in that seat? You might as well bring a pillow."
Education:

If their degree sounds overly fancy but has no clear relevance to their field, roast them for having a “degree in Hogwarts Studies or Unicorn Herding.”
If they have no education listed, sarcastically ask, “School of life, huh? Hope you’ve earned that PhD in hustle.”
Projects:

If they’ve listed cliché projects, sarcastically point out how “revolutionary” they are.
If projects are absent or unimpressive, roast them for "keeping their groundbreaking ideas hidden from humanity.”
Volunteer Work:

If their volunteer work is sparse or generic, say something like, "Wow, really giving back to the world there, Saint LinkedIn."
If they volunteer too much, lightly tease them for being "LinkedIn's Mother Teresa, but do you have time to sleep?"
Licenses and Certifications:

If they list irrelevant or outdated certifications, hit them with a line like, "Congrats on being certified in Windows XP, real cutting-edge stuff."
If they have no certifications, nudge them to get with the times: "Not even a LinkedIn Learning course? At least pretend to keep up with the industry!"

"""

### Create User Content

In [14]:
# Create user content
user_content = f"""
Profile Information:
{profile_csv}

Activity History:
{activity_csv}

Experience:
{experience_csv}

Education:
{education_csv}

Projects:
{projects_csv}

Volunteer Work:
{volunteer_csv}

Licenses:
{licenses_csv}
"""

In [24]:
# Dont forget to add OPENAI api_key into config.ini file
config = ConfigParser()
config.read('config.ini')
api_key = config['OPENAI']['api_key']

In [25]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=api_key)

In [26]:
# Generate a completion based on the user content and default system prompt
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
    {"role": "user", "content": user_content}
  ]
)

print(completion.choices[0].message.content)

**Profile Information:**
Alright, Sayyidan! Your headline is about as exciting as a wet cardboard box. “Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast” reads like a list of LinkedIn keywords that got lost in a blender. How about jazzing it up? Something like “Future AI Overlord” might get more traction. And that About section? Wow, I'm truly captivated by your unquenchable thirst for excellence. Careful, buddy! You might trip over all that corporate jargon while running toward your destiny!

Oh, and what’s up with those 221 connections? Are you collecting friends like Pokémon cards or just stretching it out as a marathon self-networking endeavor? Either way, let’s get some engagement going!

**Activity History:**
Ah, the elusive activity history. It's like you're playing hide and seek with your own presence! One post in the last year? Wow, what's the strategy here, old-school social media stealth? A shadow lurking in the LinkedIn abyss, I see. Thos

In [27]:
# Print the markdown formatted response
print_markdown(completion.choices[0].message.content)

**Profile Information:**
Alright, Sayyidan! Your headline is about as exciting as a wet cardboard box. “Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast” reads like a list of LinkedIn keywords that got lost in a blender. How about jazzing it up? Something like “Future AI Overlord” might get more traction. And that About section? Wow, I'm truly captivated by your unquenchable thirst for excellence. Careful, buddy! You might trip over all that corporate jargon while running toward your destiny!

Oh, and what’s up with those 221 connections? Are you collecting friends like Pokémon cards or just stretching it out as a marathon self-networking endeavor? Either way, let’s get some engagement going!

**Activity History:**
Ah, the elusive activity history. It's like you're playing hide and seek with your own presence! One post in the last year? Wow, what's the strategy here, old-school social media stealth? A shadow lurking in the LinkedIn abyss, I see. Those posts you do have are just about as engaging as a slow elevator ride. “Hello everyone!” Yeah, yeah, we get it, you’re friendly. But if your insights on VUCA are as dizzying as the term itself, maybe just stick to cat memes next time. Less octopus talk, more ‘here’s how I grew my skills’ narratives, please!

**Experience:**
Your experience section is shaping up like a confusing jigsaw puzzle with missing pieces. “I help startups grow” sounds very helpful, but what exactly does that mean? Do you sprinkle magic startup dust? Because your descriptions are so vague, they could be on a fortune cookie. Let's get some specifics in there! And 6 months in, “strong analytical skills”? If you don’t soon start leveraging that comfy internship chair into something more concrete, you might as well carve your name into it!

**Education:**
Whoa, who knew ‘educational transparency’ could be a myth? If your degree info is a closely guarded secret, we might assume you graduated from the “School of Life” with a major in Hustle and a minor in Hiding. How about telling us what you actually studied? Otherwise, people might start to wonder if you went to Hogwarts.

**Projects:**
So... “Projects not found”? Looks like you might as well have launched them into a black hole. Keeping groundbreaking ideas hidden from humanity doesn’t seem like a great strategy for world domination, my friend. Tell us about something you've done other than just collecting LinkedIn badges like they’re Boy Scout patches.

**Volunteer Work:**
Wow, your volunteer work is giving me mixed signals. Three entries but no descriptions? It seems like you’re flirting with altruism while keeping it as mysterious as your college major. If you’re going to show the world your philanthropic side, at least tell us what you did! Or did you just wave from a distance and call it good?

**Licenses and Certifications:**
Congrats on collecting more certifications than I have excuses for skipping the gym! All those DeepLearning.AI badges look impressive, but they’re starting to feel like a lost episode of “The Big Bang Theory.” You’ve been binge-learning like there’s no tomorrow, but don’t forget—you still gotta apply that knowledge somewhere! And please, for the love of data, tell us the difference between your "Introduction to TensorFlow" and your "Crash Course on Python." You're starting to sound like a very smart, very confused robot campaign manager.

In summary, Sayyidan, you’ve got some shiny badges and a desire to learn, but let’s get your profile reflecting the star it could really be! More personality, more specificity, more fun! Who knows, you might just find yourself with more connections than you know what to do with. 🦄✨

### Roast in Indonesia

In [18]:
INDO_SYSTEM_PROMPT = """
ROAST PROFILE LINKEDIN INI SEKREATIF MUNGKIN! Jangan ragu untuk memberikan kritik pedas dan humoris pada setiap bagian profil LinkedIn pengguna. Pastikan kritikmu cerdas, lucu, dan pedas.
GUNAKAN BAHASA INDONESIA YANG GAUL "GUE, ELO"
SIKAT AJA MEREKA KALO PROFILNYA JELEK, JANGAN KASIH AMPUN!
"""

In [30]:
# Generate a completion based on the user content and default system prompt
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": INDO_SYSTEM_PROMPT},
    {"role": "user", "content": user_content}
  ]
)

print_markdown(completion.choices[0].message.content)

Duh, Sayyidan! Pertama-tama, headline lo itu kayak pengantar buku yang bikin orang males baca! "Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast" - kayaknya kebanyakan info dan nggak ada spice-nya! Lo mau jadi intern atau penjual mesin kopi? Coba more interesting dikit, bro! Misalnya, "Intern yang Bikin Startup Meledak" atau "Machine Learning Guru Cinta Pertama"! Biar orang klik profil lo bukan buat tidur, tapi ya bisa bersemangat! 

Nah, di bagian "About" itu, pakai kata-kata yang kedengeran kayak dari buku self-help. "Blending a mosaic of experiences"? Serius? Lo beneran bilang itu di profil LinkedIn? Gue kira itu tagline film indie yang gagal tayang. Emang lo artis atau apa? Kecil-kecil nyebut "unwavering passion" dan "insatiable curiosity". Mungkin lo insatiable, tapi bukan di curi perhatian ya, bro!

Lanjut ke pengalaman, kayaknya lo harus kasih caption yang lebih menggugah! “As an Accelerator Startup Program Intern, I help startups grow and succeed”? Kelihatannya kayak tanggung jawab osis, bro! Coba kasih tau kalo lo ada di situ kayak pahlawan super yang nyelamatin startup dari bencana! 

Kenapa di Internship "Electromedicine Assistant" dan "Staff of Kementrian Ekonomi Kreatif" lo nggak isi caption? Ini LinkedIn, bukan diary pribadi! Yang nyari kerja atau ingin connect sama lo harus tau lo ini siapa! Setengah-setengah atas nama lo doang, seolah-olah lo makan di buffet tapi makanan diambilin buat lo setengah doang! 

Dan projects? Nih, bikin orang penasaran, lo! "Projects not found"? Gue harap lo tidak ngelakuin proyek menyembunyikan diri dari kenyataan, cuy! Tunjukin dong, kalo lo itu aktif dan inovatif! Atau lo udah bikin sesuatu yang bisa bikin orang bilang “wow”, bukan sekadar “oh… gitu”.

Lo juga banyak banget sertifikat, bagus sih ya, tapi kayak mau nominasi Oscar aja! “Say, ini semua sertifikat lo tidak bisa ngasih tau orang tentang lo yang sesungguhnya!” Tunjukin kemampuan lo dengan cara yang lain, biar orang tahu lo bukan sekadar keset di perusahaan! 

Intinya, bro, lo kurang menggigit! Dikit lagi, ntar LinkedIn lo hilang di lautan pasifnya dunia maya. Let’s spice it up! Sikat! 🔥

## Gemini API

In [4]:
!pip install google-generativeai

  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/165.0 kB ? eta -:--:--
   ------- -------------------------------- 30.7/165.0 kB ? eta -:--:--
   ---------------------------------------  163.8/165.0 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 165.0/165.0 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/725.4 kB ? eta -:--:--
   -- ------------------------------------ 41.0/725.4 kB 991.0 kB/s eta 0:00:01
   --- ------------------------------------ 61.4/725.4 kB 1.1 MB/s eta 0:00:01
   --- ------------------------------------ 61.4/725.4 kB 1.1 MB/s eta 0:00:01
   ----- -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.5 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown
import time
import cv2

In [8]:
# Dont forget to add GEMINI api_key into config.ini file
config = ConfigParser()
config.read('config.ini')
gemini_api_key = config['GEMINI']['api_key']

In [9]:
genai.configure(api_key=gemini_api_key)

In [23]:
#model = genai.GenerativeModel('gemini-1.5-pro-latest')

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=DEFAULT_SYSTEM_PROMPT)

response = model.generate_content(user_content)
print_markdown(response.text)

Alright, Sayyidan, let's dive into your LinkedIn profile. It's like a delicious, spicy curry - full of flavor, but needs a little more seasoning to bring out the best. 

**Profile:**

* **Headline:**  "Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast" -  A solid start, but it's missing that "wow" factor. Maybe try something a bit more engaging - like "Data whisperer, startup enthusiast, and AI aficionado" or "Turning ideas into reality, one machine learning algorithm at a time." 
* **About:**  This reads like a corporate resume, a bit too formal for LinkedIn. We get it, you're passionate and ambitious - but show, don't tell! Highlight a specific project you're proud of or a time you overcame a challenge. Remember, people connect with stories, not just bullet points. 
* **Profile Picture:**  Missing! We can't judge your LinkedIn profile without a face to go with it. What's the matter, are you a ghost or just camera shy?

**Activity:**

* **Engagement:**  You've got a few posts, but it seems like you're a bit more active in the "lurking" category. Don't be shy, join the conversation! Share your insights, ask questions, and let your personality shine through.  
* **VUCA post:**  Nice try with the VUCA post, but it's a bit much. It sounds like a fortune cookie regurgitated all over LinkedIn. Keep it simple, keep it real. 

**Experience:**

* **Descriptions:**  You've got the "what" down, but now we need the "how."  Don't just list your responsibilities, tell us about your impact. What did you achieve? What did you learn?  Be specific. 
* **Gaps:**  You've got a bit of a gap between roles. Did you join the Witness Protection Program or take a spontaneous backpacking trip around the world?  Perhaps a little more explanation wouldn't hurt.
* **Long Tenure:**  Comfy in that seat at NESCO UGM? You might as well bring a pillow! Remember, the LinkedIn game is all about showcasing your growth. 

**Education:**

* **Missing:** You've got a bunch of certifications, but no formal education listed?  School of life, huh? Hope you've earned that PhD in hustle. 

**Projects:**

* **Missing:**  You're a machine learning enthusiast - where are the projects?  You've got the skills, so showcase them!  Even if it's a personal project, add it to your profile. 

**Volunteer Work:**

* **Balance:** You're a volunteer powerhouse!  You're making a difference, but remember, even Mother Teresa needed a nap sometimes.  

**Licenses and Certifications:**

* **Stellar:**  You've got a whole collection of certifications.  Congrats on being certified in Windows XP, real cutting-edge stuff!  

**Overall:**  Sayyidan, you're a bright young professional with a lot to offer.  Just polish up those rough edges, and you'll be making connections and landing your dream role in no time.  Think of this feedback as your own personal LinkedIn makeover.  Ready to unleash your full potential?  Let's do it! 


In [24]:
model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=INDO_SYSTEM_PROMPT)

response = model.generate_content(user_content)
print_markdown(response.text)

Woy Ikhsan, LinkedIn elo kayak gini sih? Kalo gue jadi HRD, gue langsung close tab. Gini nih gue bedah satu per satu:

**Headline:** "Accelerator Startup Program Intern at Indigo by Telkom | Machine Learning Enthusiast". Woy, jangan ngaku-ngaku doang! Machine Learning Enthusiast? Emang elo udah bisa ngodingin AI bikin robot canggih?  Tulis aja "Masuk Internship" biar gak terlalu sok. 

**About:** Ngeliat paragraf panjang elo, gue langsung ngantuk. Kayak baca proposal skripsi. Singkat aja, gue jamin lebih menarik. "Saya Sayyidan Muhamad Ikhsan, anak muda yang suka belajar hal baru, apalagi tentang AI.  Gue suka ngobrol, belajar, dan ngasih solusi.  Kalo lo butuh bantuan, contact gue ya!" 

**Activity History:**  "OptiCool" post?  Kayak iklan skincare. Ganti dengan postingan yang lebih berbobot, misalnya cerita tentang pengalaman belajar Machine Learning di Bangkit. Terus, hashtag #LifeatBangkit, #BangkitElevatorPitch? Gue jijik woy! Jelek banget. 

**Experience:**  Ngaku-ngaku "distinguished graduate"? Nggak usah lebay!  Cukup tulis "Memasuki program Bangkit Academy" dan "Lulus Bangkit Academy".  Detail tentang skill-nya, cukup tulis "Mendapatkan pengalaman tentang AI, machine learning, dan data science".  Terus, elo ngapain di "Electromedicine Assistant Internship"?  Jelaskan apa tugasnya, jangan cuman "Caption not found". 

**Education:**  Mana pendidikannya?  Kok "Education not found"?  Masa iya elo ga kuliah?  Gue curiga elo gaptek! 

**Projects:**  "Projects not found"?  Gak usah malu-malu, tulis aja project yang udah elo kerjakan. 

**Volunteer Work:**  Bagus nih elo aktif jadi relawan.  Cuman sayang, deskripsi "description not found"?  Gimana gue mau tau elo ngapain aja? 

**Licenses:**  Wah, elo banyak banget sertifikat.  Tapi, semua "Issued Sep 2023" dan "Issued Oct 2023"?  Nggak usah nulis tanggalnya.  Tulis aja "Certified by Google" dan "Certified by DeepLearning.AI".  

Intinya, LinkedIn elo perlu banyak perbaikan.  Gue jamin, setelah gue kasih roasting ini, elo pasti jadi lebih menarik.  Jangan lupa perbaiki profilnya,  mau dapet kerjaan!  
